In [ ]:
import os, json, uuid
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
system_message = """You are a concise travel assistant.
Use tools when needed. Ask only one short follow-up question if required.
Return clear results and confirmations.
"""

In [ ]:
ticket_prices = {"london": 799, "paris": 899, "tokyo": 1400, "berlin": 499}
bookings = {}  

In [ ]:
def search_flights(origin: str, destination: str, date: str):
    # simple deterministic "results"
    base = ticket_prices.get(destination.lower(), 999)
    return [
        {"flight_id": f"{origin[:2].upper()}{destination[:2].upper()}-101", "origin": origin, "destination": destination, "date": date, "price_usd": base},
        {"flight_id": f"{origin[:2].upper()}{destination[:2].upper()}-202", "origin": origin, "destination": destination, "date": date, "price_usd": base + 120},
    ]

In [ ]:
def get_ticket_price(destination_city: str):
    price = ticket_prices.get(destination_city.lower())
    return {"destination_city": destination_city, "price_usd": price if price is not None else None}

In [ ]:
def book_flight(flight_id: str, passenger_name: str):
    booking_id = str(uuid.uuid4())[:8]
    booking = {"booking_id": booking_id, "flight_id": flight_id, "passenger_name": passenger_name, "status": "CONFIRMED"}
    bookings[booking_id] = booking
    return booking

In [ ]:
def get_booking_status(booking_id: str):
    return bookings.get(booking_id, {"booking_id": booking_id, "status": "NOT_FOUND"})

In [ ]:
def cancel_booking(booking_id: str):
    b = bookings.get(booking_id)
    if not b:
        return {"booking_id": booking_id, "status": "NOT_FOUND"}
    b["status"] = "CANCELLED"
    return b

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search available flights for a given route and date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},
                    "date": {"type": "string", "description": "Date in YYYY-MM-DD"},
                },
                "required": ["origin", "destination", "date"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get a typical ticket price for a destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {"type": "string"},
                },
                "required": ["destination_city"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "book_flight",
            "description": "Book a flight for a passenger using a flight_id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_id": {"type": "string"},
                    "passenger_name": {"type": "string"},
                },
                "required": ["flight_id", "passenger_name"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_booking_status",
            "description": "Get current status of an existing booking.",
            "parameters": {
                "type": "object",
                "properties": {"booking_id": {"type": "string"}},
                "required": ["booking_id"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_booking",
            "description": "Cancel an existing booking.",
            "parameters": {
                "type": "object",
                "properties": {"booking_id": {"type": "string"}},
                "required": ["booking_id"],
                "additionalProperties": False,
            },
        },
    },
]

In [ ]:
tool_impl = {
    "search_flights": search_flights,
    "get_ticket_price": get_ticket_price,
    "book_flight": book_flight,
    "get_booking_status": get_booking_status,
    "cancel_booking": cancel_booking,
}

In [ ]:
def _clean_history(history):
    return [{"role": h["role"], "content": h["content"]} for h in history]

In [ ]:
def _run_tool_call(tool_call):
    fn_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments or "{}")
    result = tool_impl[fn_name](**args)
    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result),
    }

In [ ]:
def chat(message, history):
    history = _clean_history(history)
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    resp = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while resp.choices[0].finish_reason == "tool_calls":
        assistant_msg = resp.choices[0].message
        messages.append({"role": "assistant", "content": assistant_msg.content or "", "tool_calls": assistant_msg.tool_calls})

        for tc in assistant_msg.tool_calls:
            messages.append(_run_tool_call(tc))

        resp = client.chat.completions.create(model=MODEL, messages=messages)

    return resp.choices[0].message.content

In [ ]:
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Flight Booking Assistant (Tools + Function Calling)",
    examples=[
        "Find me flights from Lagos to London on 2026-03-10.",
        "Book flight LA LO-101 for Collins Nwaogugu.",
        "Check booking status for booking id 12345678.",
        "Cancel booking 12345678.",
    ],
).launch()